In [1]:
import pandas as pd
import numpy as np

In [11]:
cleaned_data = pd.read_csv('cleaned_data.csv')
smaller_cleaned_data = cleaned_data.sample(n=2000, random_state=52)
print(smaller_cleaned_data['date'].isna().sum())  # Number of NaNs in 'date'
print(smaller_cleaned_data['time_of_day'].isna().sum())  # Number of NaNs in 'time_of_day'

/var/folders/p5/fngr6spx4xb9wrzt1z70d9k80000gn/T/ipykernel_216/3516458036.py:1: DtypeWarning: Columns (7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  cleaned_data = pd.read_csv('cleaned_data.csv')


1614
1614


In [19]:
smaller_cleaned_data['date'] = pd.to_datetime(smaller_cleaned_data['date'], errors='coerce')
smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].astype(str)
smaller_cleaned_data['date'] = smaller_cleaned_data['date'].fillna(method='ffill')
smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].fillna(method='bfill')

missing_date_rows = smaller_cleaned_data[smaller_cleaned_data['date'].isnull()]

print(smaller_cleaned_data[['date', 'time_of_day']].isnull().sum())

date           0
time_of_day    0
dtype: int64


/var/folders/p5/fngr6spx4xb9wrzt1z70d9k80000gn/T/ipykernel_216/27955149.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  smaller_cleaned_data['date'] = smaller_cleaned_data['date'].fillna(method='ffill')
/var/folders/p5/fngr6spx4xb9wrzt1z70d9k80000gn/T/ipykernel_216/27955149.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].fillna(method='bfill')


In [ ]:
# Ensure time_begin is in datetime format
smaller_cleaned_data['time_begin'] = pd.to_datetime(smaller_cleaned_data['time_begin'], errors='coerce')

# Fill missing 'date' values with the forward fill (ffill), or use a specific date (e.g., '2020-01-01') if needed
smaller_cleaned_data['date'] = pd.to_datetime(smaller_cleaned_data['date'], errors='coerce')
smaller_cleaned_data['date'] = smaller_cleaned_data['date'].fillna(method='ffill')

# Fill missing 'time_of_day' values with backward fill (bfill), or use forward fill (ffill) if needed
smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].astype(str)
smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].fillna(method='bfill')

# Fill missing 'time_end' by adding 1 hour to 'time_begin' if missing
smaller_cleaned_data['time_end'] = smaller_cleaned_data['time_end'].fillna(smaller_cleaned_data['time_begin'] + pd.Timedelta(hours=1))

# Check if there are still any missing values in the relevant columns
missing_values = smaller_cleaned_data[['date', 'time_of_day', 'time_end']].isnull().sum()

# Ensure 'time_begin' is in datetime format
smaller_cleaned_data['time_begin'] = pd.to_datetime(smaller_cleaned_data['time_begin'], errors='coerce')

# Forward-fill missing 'time_begin' values (or use bfill depending on your needs)
smaller_cleaned_data['time_begin'] = smaller_cleaned_data['time_begin'].fillna(method='ffill')

# Convert 'time_of_day' to timedelta (time in minutes from midnight)
smaller_cleaned_data['time_of_day_minutes'] = pd.to_datetime(smaller_cleaned_data['time_of_day'], format='%H:%M', errors='coerce').dt.hour * 60 + pd.to_datetime(smaller_cleaned_data['time_of_day'], format='%H:%M', errors='coerce').dt.minute

# Calculate the mean of 'time_of_day' (in minutes)
mean_time_minutes = smaller_cleaned_data['time_of_day_minutes'].mean()

# Convert the mean back to a time string (format HH:MM)
mean_time = pd.to_datetime(str(int(mean_time_minutes // 60)) + ':' + str(int(mean_time_minutes % 60)), format='%H:%M').strftime('%H:%M')

# Fill missing 'time_of_day' with the mean time
smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].fillna(mean_time)

# Optional: Drop the temporary column used for calculating mean
smaller_cleaned_data.drop(columns=['time_of_day_minutes'], inplace=True)

smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].replace('nan', )

# Print the result to verify
print("Missing values after cleaning:")
print(missing_values)

# Optionally, print the first few rows of the cleaned data to inspect the result
print(smaller_cleaned_data[['date', 'time_of_day', 'time_end']].head())


/var/folders/p5/fngr6spx4xb9wrzt1z70d9k80000gn/T/ipykernel_216/4257835832.py:6: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  smaller_cleaned_data['date'] = smaller_cleaned_data['date'].fillna(method='ffill')
/var/folders/p5/fngr6spx4xb9wrzt1z70d9k80000gn/T/ipykernel_216/4257835832.py:10: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].fillna(method='bfill')
/var/folders/p5/fngr6spx4xb9wrzt1z70d9k80000gn/T/ipykernel_216/4257835832.py:22: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  smaller_cleaned_data['time_begin'] = smaller_cleaned_data['time_begin'].fillna(method='ffill')


Missing values after cleaning:
date           0
time_of_day    0
time_end       0
dtype: int64
            date time_of_day             time_end
Index                                            
1758  2020-01-01       20:00  2020-06-08 10:10:00
2403  2020-01-01       20:00  2020-03-06 10:00:00
759   2020-02-19       20:00                20:30
1479  2019-10-19       12:10                12:20
366   2019-10-21       10:30                10:37


In [ ]:
# Fill in time of day as something random between 00:00 and 23:59, inclusive
##################

import random

# Function to generate a random time in the format 'HH:MM'
def random_time():
    # Randomly choose hours and minutes
    hour = random.randint(0, 11)  # Random hour between 0 and 11 (inclusive)
    minute = random.randint(0, 59)  # Random minute between 0 and 59
    return f"{hour:02}:{minute:02}"  # Format the time as 'HH:MM'

# Replace 'nan' strings with a random time throughout the day
smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].replace('nan', np.nan)

# Replace NaN (or missing) values in 'time_of_day' with a random time
smaller_cleaned_data['time_of_day'] = smaller_cleaned_data['time_of_day'].apply(
    lambda x: random_time() if pd.isna(x) else x
)

# Check the result
print(smaller_cleaned_data[['time_of_day']].head())


      time_of_day
Index            
1758        20:00
2403        20:00
759         20:00
1479        12:10
366         10:30


In [43]:
smaller_cleaned_data

,timestamp_x,Event Type,Source Device ID,glucose_level,Transmitter Time (Long Integer),participant_id,date,time_of_day,time_begin,time_end,...,calorie,total_carb,dietary_fiber,sugar,protein,total_fat,time,timestamp_y,Gender,HbA1c
Index,,,,,,,,,,,,,,,,,,,,,
1758,2020-06-11 17:47:59,EGV,Android G6,97.0,531288.0,14,2020-01-01,20:00,2020-06-08 09:10:00,2020-06-08 10:10:00,...,120.0,19.0,0.0,9.0,1.0,4.5,09:10:00,2020-02-21 12:00:00,MALE,5.5
2403,2020-03-06 21:56:08,EGV,MX84702427,121.0,731459.0,5,2020-01-01,20:00,2020-03-06 09:00:00,2020-03-06 10:00:00,...,11.0,2.7,0.5,1.0,0.4,0.1,09:00:00,2020-02-21 12:00:00,FEMALE,5.7
759,2020-07-23 07:38:50,EGV,iPhone G6,114.0,1532692.0,15,2020-02-19,20:00,2020-02-19 20:00:00,20:30,...,170.0,27.0,1.0,13.0,1.0,6.0,12:00:00,2020-02-19 12:00:00,FEMALE,5.5
1479,2020-03-19 21:32:04,EGV,Android G6,73.0,473088.0,7,2019-10-19,12:10,2020-02-19 20:00:00,12:20,...,35.0,8.0,7.0,0.0,0.0,0.0,12:00:00,2019-10-19 12:00:00,FEMALE,5.3
366,2020-03-15 18:12:16,EGV,Android G6,75.0,115498.0,7,2019-10-21,10:30,2020-02-19 20:00:00,10:37,...,280.0,56.5,1.0,24.0,8.0,2.5,12:00:00,2019-10-21 12:00:00,FEMALE,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260,2020-03-06 10:01:07,EGV,MX84702427,103.0,688556.0,5,2020-02-22,11:21,2020-02-27 15:45:00,2020-02-27 16:45:00,...,24.0,5.9,0.0,3.8,0.0,0.0,15:45:00,2020-02-21 12:00:00,FEMALE,5.7
1273,2020-03-27 03:04:10,EGV,iPhone G6,90.0,410087.0,10,2020-02-22,04:57,2020-03-23 18:05:00,2020-03-23 19:05:00,...,7.8,1.9,0.3,0.9,0.3,0.1,18:05:00,2020-02-21 12:00:00,FEMALE,6.0
54,2020-03-14 17:34:52,EGV,Android G6,76.0,20700.0,8,2020-02-22,00:35,2020-03-15 16:50:00,17:25:00,...,324.0,11.0,0.3,0.0,20.0,22.0,16:50:00,2020-02-21 12:00:00,FEMALE,5.6


In [44]:
# Convert timestamps to datetime format
smaller_cleaned_data["timestamp_x"] = pd.to_datetime(smaller_cleaned_data["timestamp_x"])
smaller_cleaned_data["timestamp_y"] = pd.to_datetime(smaller_cleaned_data["timestamp_y"])
smaller_cleaned_data["time_begin"] = pd.to_datetime(smaller_cleaned_data["time_begin"])
smaller_cleaned_data["time_end"] = pd.to_datetime(smaller_cleaned_data["time_end"])
smaller_cleaned_data["date"] = pd.to_datetime(smaller_cleaned_data["date"])

# Decide on a single timestamp column (choosing `timestamp_x` if it's primary)
smaller_cleaned_data["timestamp"] = smaller_cleaned_data["timestamp_x"].combine_first(smaller_cleaned_data["timestamp_y"])

# Drop unnecessary duplicates
smaller_cleaned_data.drop(["timestamp_x", "timestamp_y"], axis=1, inplace=True)

# Fill missing time-related values
smaller_cleaned_data["time_of_day"] = smaller_cleaned_data["time_of_day"].fillna(smaller_cleaned_data["time"])  # Merge redundant columns

# Sort by timestamp
smaller_cleaned_data = smaller_cleaned_data.sort_values("timestamp")

In [45]:
smaller_cleaned_data

,Event Type,Source Device ID,glucose_level,Transmitter Time (Long Integer),participant_id,date,time_of_day,time_begin,time_end,logged_food,...,calorie,total_carb,dietary_fiber,sugar,protein,total_fat,time,Gender,HbA1c,timestamp
Index,,,,,,,,,,,,,,,,,,,,,
39,EGV,iPhone G6,110.0,18901.0,1,2020-02-21,12:10,2020-02-20 07:37:00,2020-02-20 08:37:00,Bagel,...,139.0,27.0,1.639904,4.4,5.5,8.057602,07:37:00,FEMALE,5.5,2020-02-13 19:33:32
44,EGV,iPhone G6,124.0,20401.0,1,2020-02-17,13:15,2020-02-21 07:30:00,2020-02-21 08:30:00,Standard Breakfast,...,110.0,26.0,1.639904,10.0,1.0,8.057602,07:30:00,FEMALE,5.5,2020-02-13 19:58:31
65,EGV,iPhone G6,91.0,26701.0,1,2019-10-20,16:15,2020-02-17 21:00:00,2020-02-17 22:00:00,Kale Salad,...,83.7,4.0,1.639904,2.1,1.2,8.057602,21:00:00,FEMALE,5.5,2020-02-13 21:43:32
68,EGV,iPhone G6,89.0,27601.0,1,2020-02-22,19:05,2020-02-17 12:30:00,2020-02-17 13:30:00,Salad with Cranberries,...,80.0,21.0,1.639904,17.0,0.0,8.057602,12:30:00,FEMALE,5.5,2020-02-13 21:58:31
142,EGV,iPhone G6,91.0,49800.0,1,2019-10-21,20:30,2020-02-18 06:20:00,2020-02-18 07:20:00,Natrel Lactose Free 2 Percent,...,120.0,9.0,1.639904,8.0,12.0,8.057602,06:20:00,FEMALE,5.5,2020-02-14 04:08:31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1617,EGV,iPhone G6,122.0,1868690.0,15,2020-02-21,17:00,2020-02-21 17:00:00,2020-02-21 18:00:00,Small chocolate shake (1/2),...,270.0,48.0,0.700000,47.3,7.0,6.300000,12:00:00,FEMALE,5.5,2020-07-27 04:58:46
1631,EGV,iPhone G6,122.0,1872890.0,15,2020-02-20,18:38,2020-02-20 18:38:00,2025-02-18 18:40:00,Banana,...,105.0,27.0,3.000000,14.5,1.3,0.400000,12:00:00,FEMALE,5.5,2020-07-27 06:08:46
1633,EGV,iPhone G6,120.0,1873490.0,15,2020-02-19,20:00,2020-02-19 20:00:00,2025-02-18 20:30:00,Taco Bell Nachos Grande,...,740.0,82.0,15.000000,5.0,16.0,38.000000,12:00:00,FEMALE,5.5,2020-07-27 06:18:46
